In [28]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [26]:
lsof1 = pd.DataFrame()
# because of space in Name field, had to do this work around
# ideally it should have worked without setting columns, if there were no spaces in NAME column
cols=['COMMAND' ,'PID', 'USER', 'FD' ,'TYPE' ,'DEVICE' \
               ,'SIZE/OFF' ,'NODE', 'NETWORK_INFO' ,'CONN_STATUS']
lsof1 = pd.read_csv('data/lsof_07052020_1.txt',sep="\s+",\
                    header=None,skiprows=1,names=cols)
tcp = lsof1.groupby('NODE').get_group('TCP')
# remove connection with status as LISTEN, these only contain localhost and port
tcp_Est = tcp[tcp['CONN_STATUS']!='(LISTEN)']

# extracting TCP to and from information
tcp_Est['FROM_ADDR'] = tcp_Est['NETWORK_INFO'].str.split('->').str.get(0)
tcp_Est['TO_ADDR'] = tcp_Est['NETWORK_INFO'].str.split('->').str.get(1)
# rsplit is used since IPV6 has colon symbol in the address, 
# hence using reverse split
tcp_Est['TO_PORT'] = tcp_Est['TO_ADDR'].str.rsplit(':',n=1).str.get(1)
tcp_Est['TO_IP'] = tcp_Est['TO_ADDR'].str.rsplit(':',n=1).str.get(0)
tcp_Est['FROM_PORT'] = tcp_Est['FROM_ADDR'].str.rsplit(':',n=1).str.get(1)
tcp_Est['FROM_IP'] = tcp_Est['FROM_ADDR'].str.rsplit(':',n=1).str.get(0)
# dropping unnecessary columns
tcp_Est.drop('TO_ADDR',axis=1,inplace=True)
tcp_Est.drop('FROM_ADDR',axis=1,inplace=True)

#summary by grouping from/to ips and connection status
ip_summary = pd.DataFrame(tcp_Est.groupby(['PID','COMMAND','FROM_IP','TO_IP','CONN_STATUS','TYPE']).COMMAND.count()).\
   rename(columns={'COMMAND':'COUNT'}).sort_values('COUNT',ascending=False)
ip_summary

COUNT
PID   COMMAND   FROM_IP                                            TO_IP                                              CONN_STATUS   TYPE       
2377  python3.8 localhost                                          localhost                                          (ESTABLISHED) IPv4     12
21309 python3.8 localhost                                          localhost                                          (ESTABLISHED) IPv4      6
2387  python3.8 localhost                                          localhost                                          (ESTABLISHED) IPv4      6
2377  python3.8 localhost                                          localhost                                          (ESTABLISHED) IPv6      3
17859 Google    localhost                                          localhost                                          (ESTABLISHED) IPv6      3
505   identitys vishalendus-macbook-pro.local                      [fe80:10::c06a:ecd4:d425:d206]                     (ESTABLISHED) IPv6      3
1612  Microsoft 192.168.1.9                                        52.109.12.9                                        (ESTABLISHED) IPv4      2
17859 Google    2603-6000-b700-0034-3011-b90e-ac6f-45af.res6.sp... ie-in-xbc.1e100.net                                (ESTABLISHED) IPv6      2
                                                                   [2620:109:c002::6cae:a0a]                          (ESTABLISHED) IPv6      2
                192.168.1.9                                        ec2-35-174-127-31.compute-1.amazonaws.com          (ESTABLISHED) IPv4      2
                2603-6000-b700-0034-3011-b90e-ac6f-45af.res6.sp... [2606:4700:10::6814:30a6]                          (ESTABLISHED) IPv6      1
1612  Microsoft 192.168.1.9                                        104.42.252.157                                     (ESTABLISHED) IPv4      1
17859 Google    2603-6000-b700-0034-3011-b90e-ac6f-45af.res6.sp... ord37s19-in-x05.1e100.net                          (ESTABLISHED) IPv6      1
                                                                   ord36s01-in-x05.1e100.net                          (ESTABLISHED) IPv6      1
                                                                   edge-star6-shv-01-ort2.facebook.com                (ESTABLISHED) IPv6      1
                                                                   [2a04:4e42:3e::343]                                (ESTABLISHED) IPv6      1
                                                                   [2620:108:700f::3422:9ea1]                         (ESTABLISHED) IPv6      1
                                                                   [2620:108:700f::22df:8202]                         (ESTABLISHED) IPv6      1
                                                                   [2606:4700::6812:1bd3]                             (ESTABLISHED) IPv6      1
                                                                   [2606:4700::6812:1ad3]                             (ESTABLISHED) IPv6      1
                                                                   [2606:4700:10::6814:2fa6]                          (ESTABLISHED) IPv6      1
                192.168.1.9                                        104.16.0.35                                        (ESTABLISHED) IPv4      1
                                                                   server-52-85-90-214.ord53.r.cloudfront.net         (ESTABLISHED) IPv4      1
1609  WhatsApp  2603-6000-b700-0034-3011-b90e-ac6f-45af.res6.sp... whatsapp-cdn6-shv-01-ort2.fbcdn.net                (ESTABLISHED) IPv6      1
17859 Google    192.168.1.9                                        customer.clientshostname.com                       (SYN_SENT)    IPv4      1
                                                                                                                      (ESTABLISHED) IPv4      1
                                                                   a23-65-241-243.deploy.static.akamaitech

In [27]:
FILTER = ip_summary.reset_index()['TYPE'] == 'IPv4'
ip_summary.reset_index()[FILTER]

,PID,COMMAND,FROM_IP,TO_IP,CONN_STATUS,TYPE,COUNT
0,2377,python3.8,localhost,localhost,(ESTABLISHED),IPv4,12
1,21309,python3.8,localhost,localhost,(ESTABLISHED),IPv4,6
2,2387,python3.8,localhost,localhost,(ESTABLISHED),IPv4,6
6,1612,Microsoft,192.168.1.9,52.109.12.9,(ESTABLISHED),IPv4,2
9,17859,Google,192.168.1.9,ec2-35-174-127-31.compute-1.amazonaws.com,(ESTABLISHED),IPv4,2
11,1612,Microsoft,192.168.1.9,104.42.252.157,(ESTABLISHED),IPv4,1
21,17859,Google,192.168.1.9,104.16.0.35,(ESTABLISHED),IPv4,1
22,17859,Google,192.168.1.9,server-52-85-90-214.ord53.r.cloudfront.net,(ESTABLISHED),IPv4,1
24,17859,Google,192.168.1.9,customer.clientshostname.com,(SYN_SENT),IPv4,1
25,17859,Google,192.168.1.9,customer.clientshostname.com,(ESTABLISHED),IPv4,1
